# Book Rcommender System

In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [4]:
# Load data
books = pd.read_csv('Books.csv')
ratings = pd.read_csv('Ratings.csv')
users = pd.read_csv('Users.csv')

C:\Users\71524\AppData\Local\Temp\ipykernel_6532\4250405617.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


In [5]:
# Preprocess data
ratings = ratings[ratings['Book-Rating'] > 0]  # Filter out zero ratings since it is unrated
book_ratings = pd.merge(ratings, books, on='ISBN')  # Merge with book info

In [ ]:
# Check how many unique book titles are present
unique_titles_count = book_ratings['Book-Title'].nunique()
print(f"Number of unique book titles: {unique_titles_count}")

Number of unique book titles: 135567


In [32]:
# Filter for books rated more than 5 times
book_ratings_filtered = book_ratings.groupby('Book-Title').filter(lambda x: len(x) > 5)
print(f"Filtered data has {book_ratings_filtered['Book-Title'].nunique()} unique book titles.")

Filtered data has 10844 unique book titles.


In [39]:
# Create pivot table
rating_pivot = book_ratings_filtered.pivot_table(index='User-ID', columns='Book-Title', values='Book-Rating').fillna(0)

# Check if there are any non-zero ratings
non_zero_ratings = book_ratings[book_ratings['Book-Rating'] > 0]
print(non_zero_ratings.shape)  # This will give the number of rows with non-zero ratings
print(non_zero_ratings.head())  # This will show the first few rows with non-zero ratings


(383842, 10)
   User-ID        ISBN  Book-Rating  \
0   276726  0155061224            5   
1   276729  052165615X            3   
2   276729  0521795028            6   
3   276744  038550120X            7   
4   276747  0060517794            9   

                                          Book-Title    Book-Author  \
0                                   Rites of Passage     Judith Rae   
1                                     Help!: Level 1  Philip Prowse   
2  The Amsterdam Connection : Level 4 (Cambridge ...    Sue Leather   
3                                    A Painted House   JOHN GRISHAM   
4                           Little Altars Everywhere  Rebecca Wells   

  Year-Of-Publication                   Publisher  \
0                2001                      Heinle   
1                1999  Cambridge University Press   
2                2001  Cambridge University Press   
3                2001                   Doubleday   
4                2003                 HarperTorch   

      

In [42]:
# Convert to sparse matrix for efficiency
rating_matrix = csr_matrix(rating_pivot.values)

In [43]:
# Compute cosine similarity
user_similarity = cosine_similarity(rating_matrix)

In [44]:
# Recommend books for a given user
def recommend_books(user_id, n=5):
    if user_id not in rating_pivot.index:
        return "User not found."
    user_idx = rating_pivot.index.get_loc(user_id)
    similarity_scores = list(enumerate(user_similarity[user_idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:n+1]
    similar_users = [rating_pivot.index[i[0]] for i in similarity_scores]
    books_rated_by_similar_users = book_ratings[book_ratings['User-ID'].isin(similar_users)]
    recommended_books = (
        books_rated_by_similar_users.groupby('Book-Title')['Book-Rating']
        .mean()
        .sort_values(ascending=False)
        .head(n)
        .index
    )
    return recommended_books.tolist()

In [47]:
# Example usage
user_id = 274061
recommendations = recommend_books(user_id, n=5)
# Print recommendations under each other with numbering
print(f"Recommended Books for user {user_id}:")
for i, book in enumerate(recommendations, 1):
    print(f"{i}. {book}")

Recommended Books for user 274061:
1. 1984 (Everyman's Library)
2. Teddy Bear for Sale (Hello Reader!, Level 1)
3. Scotland Bc: An Introduction to the Prehistoric Houses, Tombs, Ceremonial Monuments, and Fortifications in the Care of the Secretary of State for Scotland (Historic Buildings and Monuments)
4. SIXTH COLUMN
5. Roller Skates! (Hello Reader! Level 2 (Paperback))
